In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Dense, Dropout, BatchNormalization,
    RandomFlip, RandomRotation, RandomZoom,
    RandomContrast, RandomBrightness,
    Rescaling, GlobalAveragePooling2D
)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import image_dataset_from_directory

# 1. إعدادات المسارات والمقاسات
DATA_DIR = r"C:\Users\baraa\Desktop\project\Alzheimer (Preprocessed Data)\Alzheimer (Preprocessed Data)"
IMAGE_SIZE = (224, 224) # مقاس ResNet50 القياسي
BATCH_SIZE = 32

# 2. تحميل بيانات التدريب
train_ds = image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
   label_mode='categorical'

)

# 3. تحميل بيانات التحقق (Validation)
val_ds = image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# 4. استخراج أسماء الفئات للتأكد
class_names = train_ds.class_names
print(f"الفئات الموجودة: {class_names}")
NUM_CLASSES = len(class_names)
print("عدد التصنيفات:", NUM_CLASSES)


In [ ]:
data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1),
    RandomContrast(0.1),
    RandomBrightness(0.1)
])

# تطبيق التحسينات لتسريع القراءة من الذاكرة
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers # هذا السطر هو الحل لمشكلة NameError
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam

# 1. تحميل ResNet50 بأوزان ImageNet
base_model = ResNet50(
    weights='imagenet', 
    include_top=False, 
    input_shape=(224, 224, 3)
)

# 2. تجميد أول 100 طبقة وفتح الباقي للتعلم (Fine-tuning)
# هذا الجزء هو اللي بيخلي الدقة تطلع فوق الـ 50%
for layer in base_model.layers[:100]:
    layer.trainable = False
for layer in base_model.layers[100:]:
    layer.trainable = True

# 3. تجميع أجزاء النموذج
model = Sequential([
    data_augmentation,         
    layers.Rescaling(1./255),  
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),      
    Dropout(0.5),              
    Dense(NUM_CLASSES, activation='softmax')

])

# 4. إعداد عملية التعلم بمعدل تعلم صغير جداً
model.compile(
    optimizer=Adam(learning_rate=3e-5), 
     loss='categorical_crossentropy',
    metrics=['accuracy']
)
# عرض ملخص النموذج
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# 1. إعداد المراقبين (Callbacks)
# التوقف المبكر: إذا لم يتحسن النموذج لـ 7 دورات سيتوقف التدريب ويأخذ أفضل نسخة
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=7, 
    restore_best_weights=True
)

# تقليل معدل التعلم: إذا ثبتت الدقة، سيقوم بتقليل "قوة التعلم" ليصبح النموذج أدق
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, # يقلل التعلم لـ 20% من قيمته
    patience=3, 
    min_lr=1e-7
)

# 2. بدء التدريب
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
# حفظ النموذج بالصيغة الحديثة
model.save('alzheimer_resnet50_final2.keras')
print("Saved successfully in .keras format!")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# 1) جمع الـ true والـ pred
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images, verbose=0)  # (batch, 4)

    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# 2) Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=class_names, yticklabels=class_names
)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

# 3) Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))
